In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Run the train script for each value of k
root = "/Users/michaeldoherty"  # /home/uceedoh
data_directory = f"{root}/git/XLRON/data/JOCN_SI/heuristic_benchmarks"
script_path = f"{root}/git/XLRON/xlron/train/train.py"

In [2]:
def read_rmsa_files(topology, heuristic, k, load, weighted=False):
    # Choice of traffic load - should be sufficient to cover wide range of blocking probs from <1% to 10%
    weight = "_weighted" if weighted else ""
    output_file = f"{data_directory}/kpaths/rmsa/{topology}/{heuristic}{weight}_k{k}_{load}.csv"
    df = pd.read_csv(output_file)
    return df


def read_rwa_lightpath_reuse_files(topology, heuristic, k, num_requests, first_blocking=False, weighted=False):
    weight = "_weighted" if weighted else ""
    output_file = f"{data_directory}/kpaths/rwa_lightpath_reuse/{topology}/{heuristic}{weight}_k{k}_{num_requests:.0f}{'_firstblocking' if first_blocking else ''}.csv"
    df = pd.read_csv(output_file)
    return df
             

In [3]:
# Data values for reference
env_type = "rmsa"
topologies = ["nsfnet_deeprmsa", "cost239_deeprmsa"]
heuristics = ['ksp_ff', 'ff_ksp', "ksp_bf", "bf_ksp", "kme_ff", "kmc_ff", "kmf_ff", "kca_ff"]
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
# Choice of traffic load - should be sufficient to cover wide range of blocking probs from <1% to 10%

env_type = "rwa_lightpath_reuse"
topologies = ["cost239", "nsfnet"]
heuristics = ['ksp_ff', 'ff_ksp', 'ksp_mu']#, 'ksp_mu_nonrel', 'ksp_mu_unique', 'mu_ksp', 'mu_ksp_nonrel', 'mu_ksp_unique']
num_requests_nsfnet = [1e4, 1e4, 1.5e4, 2e4, 2.5e4]
num_requests_cost239 = [2e4, 2e4, 2.5e4, 3e4, 3.5e4]
k_range = range(1, 11)

In [4]:
topologies = ["nsfnet_deeprmsa", "cost239_deeprmsa"]
heuristics = ['ksp_ff', 'ff_ksp', "ksp_bf", "bf_ksp", "kme_ff", "kmc_ff", "kmf_ff", "kca_ff"]
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
tables = []
for topology in topologies:
    rows = []
    for heuristic in heuristics:
        row = []
        for k in k_range:
            for load in load_range:
                df = read_rmsa_files(topology, heuristic, k, load)
                service_blocking_probability_mean = df["service_blocking_probability"].mean()
                # Get mean variance and take sqrt
                service_blocking_probability_std = np.sqrt(np.sum(df["service_blocking_probability_std"]**2)/len(df))
                row.append(service_blocking_probability_mean)
                row.append(service_blocking_probability_std)
        rows.append(row)
    table = pd.DataFrame(rows, columns=pd.MultiIndex.from_product([k_range, load_range, ["mean", "std"]], names=['k', 'load', 'BP']), index=pd.Index(heuristics, name='Heuristic'))
    tables.append(table)

# Best heuristic for each load
for i, table in enumerate(tables):
    topology = topologies[i]
    best_heuristics_all_loads = []
    for load in load_range:
        result_mean = table.loc[:, (slice(None), load, 'mean')].droplevel(['load', 'BP'], axis=1)
        result_std = table.loc[:, (slice(None), load, 'std')].droplevel(['load', 'BP'], axis=1)
        
        # Find the minimum mean value
        min_value = result_mean.min().min()
        
        # Find the index (heuristic) and column (k) of the minimum mean value
        min_index, min_column = result_mean.stack().idxmin()
        
        # Get the corresponding std value
        std_value = result_std.loc[min_index, min_column]
        
        print(f"Load: {load}")
        print(f"Minimum mean value: {min_value:.6f}")
        print(f"Corresponding std value: {std_value:.6f}")
        print(f"Heuristic: {min_index}")
        print(f"k value: {min_column}")
        
        # Find similar performing configurations
        lower_bound = min_value - std_value
        upper_bound = min_value + std_value
        
        similar_configs = result_mean[(result_mean >= lower_bound) & (result_mean <= upper_bound)]
        similar_configs = similar_configs.stack().reset_index()
        similar_configs.columns = ['Heuristic', 'k', 'BP_mean']
        
        # Get corresponding std values
        similar_configs['BP_std'] = similar_configs.apply(lambda row: result_std.loc[row['Heuristic'], row['k']], axis=1)
        
        similar_configs = similar_configs.sort_values('BP_mean')
        
        print("\nSimilar performing configurations:")
        best_hk = []
        for _, row in similar_configs.iterrows():
            hk = f"{row['Heuristic']}-{row['k']}"
            best_hk.append(hk)
            print(f"Heuristic: {row['Heuristic']}, k: {row['k']}, BP mean: {row['BP_mean']:.6f} +/- {row['BP_std']:.6f}")
        print("Similar heuristics: ", similar_configs['Heuristic'].unique())
        best_heuristics_all_loads.append(best_hk)
        
        print("\n" + "-"*50 + "\n")
    print("\n" + "="*50 + "\n")
    for heuristic in heuristics:
        for k in k_range:
            hk = f"{heuristic}-{k}"
            if all(hk in best_hk for best_hk in best_heuristics_all_loads):
                print(f"Best performing heuristic for all loads for {topology}: {hk}")



In [5]:
topologies = ["nsfnet_deeprmsa"]
heuristics = ['ksp_ff']
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
tables = []
for topology in topologies:
    rows = []
    for heuristic in heuristics:
        for k in k_range:
            row = []
            for load in load_range:
                df = read_rmsa_files(topology, heuristic, k, load, weighted=False)
                service_blocking_probability_mean = df["service_blocking_probability"].mean()
                # Get mean variance and take sqrt
                service_blocking_probability_std = np.sqrt(np.sum(df["service_blocking_probability_std"]**2)/len(df))
                row.append(service_blocking_probability_mean)
                #row.append(service_blocking_probability_std)
            rows.append(row)

df_unweighted = pd.DataFrame(rows)
        
for topology in topologies:
    rows = []
    for heuristic in heuristics:
        for k in k_range:
            row = []
            for load in load_range:
                df = read_rmsa_files(topology, heuristic, k, load, weighted=True)
                service_blocking_probability_mean = df["service_blocking_probability"].mean()
                # Get mean variance and take sqrt
                service_blocking_probability_std = np.sqrt(np.sum(df["service_blocking_probability_std"]**2)/len(df))
                row.append(service_blocking_probability_mean)
                #row.append(service_blocking_probability_std)
            rows.append(row)
               
df_weighted = pd.DataFrame(rows)
df_unweighted

In [6]:
# ((df_weighted - df_unweighted)/ df_unweighted)
# column_names = ["50", "100", "150", "200", "250", "300", "400", "500", "600", "800"]
# df_diff = ((df_weighted - df_unweighted)/ df_unweighted)
# df_diff.columns = column_names
# index = ["k=1", "k=2", "k=3", "k=4", "k=5", "k=6", "k=7", "k=8", "k=9", "k=10"]
# df_diff.index = index
# df_diff = df_diff.drop(columns=["50"])
# # Format values as percent
# df_diff.style.format("{:.2%}")
# # Format color based on value. Use a color scale where green is for 0 and red is for anything over 0.1
# df_diff.style.background_gradient(cmap='RdYlGn', axis=None).format("{:.0%}")
import pandas as pd
import numpy as np

# Assuming df_weighted and df_unweighted are already defined
column_names = ["50", "100", "150", "200", "250", "300", "400", "500", "600", "800"]
df_diff = ((df_weighted - df_unweighted) / df_unweighted)
df_diff.columns = column_names
index = ["k=1", "k=2", "k=3", "k=4", "k=5", "k=6", "k=7", "k=8", "k=9", "k=10"]
df_diff.index = index
df_diff = df_diff.drop(columns=["50"])

df_abs = (df_weighted - df_unweighted) 
df_abs.columns = column_names
index = ["k=1", "k=2", "k=3", "k=4", "k=5", "k=6", "k=7", "k=8", "k=9", "k=10"]
df_abs.index = index

# Create a custom colormap
def custom_cmap_diff(val):
    if val < 0:
        return f'background-color: rgba(0, 255, 0, {min(-val, 0.5)})'
    else:
        return f'background-color: rgba(255, 0, 0, {min(val, 0.5)})'

# Create a custom colormap
def custom_cmap_abs(val):
    if val < 0:
        return f'background-color: rgba(0, 255, 0, {min(-val, 0.5)})'
    else:
        return f'background-color: rgba(255, 0, 0, {min(val, 0.5)})'


# Apply styling
styled_df_diff = df_diff.style.applymap(custom_cmap_diff).format("{:.2%}")
styled_df_abs = df_abs.style.applymap(custom_cmap_diff).format("{:.2%}")

# Display the styled DataFrame
styled_df_diff
styled_df_abs

In [7]:
# # Best k value for each load
# for i, table in enumerate(tables):
#     topology = topologies[i]
#     best_k_all_loads = []
#     for k in k_range:
#         result_mean = table.loc[:, (slice(None), load, 'mean')].droplevel(['load', 'BP'], axis=1)
#         result_std = table.loc[:, (slice(None), load, 'std')].droplevel(['load', 'BP'], axis=1)
#         
#         # Find the minimum mean value
#         min_value = result_mean.min().min()
#         
#         # Find the index (heuristic) and column (k) of the minimum mean value
#         min_index, min_column = result_mean.stack().idxmin()
#         
#         # Get the corresponding std value
#         std_value = result_std.loc[min_index, min_column]
#         
#         print(f"Load: {load}")
#         print(f"Minimum mean value: {min_value:.6f}")
#         print(f"Corresponding std value: {std_value:.6f}")
#         print(f"Heuristic: {min_index}")
#         print(f"k value: {min_column}")
#         
#         # Find similar performing configurations
#         lower_bound = min_value - std_value
#         upper_bound = min_value + std_value
#         
#         similar_configs = result_mean[(result_mean >= lower_bound) & (result_mean <= upper_bound)]
#         similar_configs = similar_configs.stack().reset_index()
#         similar_configs.columns = ['Heuristic', 'k', 'BP_mean']
#         
#         # Get corresponding std values
#         similar_configs['BP_std'] = similar_configs.apply(lambda row: result_std.loc[row['Heuristic'], row['k']], axis=1)
#         
#         similar_configs = similar_configs.sort_values('BP_mean')
#         
#         print("\nSimilar performing configurations:")
#         for _, row in similar_configs.iterrows():
#             print(f"Heuristic: {row['Heuristic']}, k: {row['k']}, BP mean: {row['BP_mean']:.6f} +/- {row['BP_std']:.6f}")
#         print("Similar heuristics: ", similar_configs['Heuristic'].unique())
#         best_heuristics_all_loads.append(similar_configs['Heuristic'].unique())
#         
#         print("\n" + "-"*50 + "\n")
#     print("\n" + "="*50 + "\n")
#     for heuristic in heuristics:
#         if all(heuristic in best_heuristics for best_heuristics in best_heuristics_all_loads):
#             print(f"Best performing heuristic for all loads for {topology}: {heuristic}")


In [8]:
import pandas as pd
import numpy as np

def format_heuristic(heuristic):
    return heuristic.upper().replace('_', '-')

topologies = ["nsfnet_deeprmsa", "cost239_deeprmsa"]
heuristics = ['ksp_ff', 'ff_ksp', "ksp_bf", "bf_ksp", "kme_ff", "kmc_ff", "kmf_ff", "kca_ff"]
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
tables = []

for topology in topologies:
    data = []
    for heuristic in heuristics:
        for k in k_range:
            row = []
            for load in load_range:
                df = read_rmsa_files(topology, heuristic, k, load)
                service_blocking_probability_mean = df["service_blocking_probability"].mean()
                service_blocking_probability_std = np.sqrt(np.sum(df["service_blocking_probability_std"]**2)/len(df))
                row.extend([service_blocking_probability_mean, service_blocking_probability_std])
            data.append([format_heuristic(heuristic), k] + row)
    
    # Create multi-index DataFrame with titled columns
    columns = ['Heuristic', 'k']
    for load in load_range:
        columns.extend([(load, 'BP Mean'), (load, 'BP Std')])
    
    table = pd.DataFrame(data, columns=columns)
    table = table.set_index(['Heuristic', 'k'])
    
    # Create a MultiIndex for the columns
    table.columns = pd.MultiIndex.from_tuples(table.columns, names=['Load', 'Metric'])
    
    tables.append(table)

# Example of how to display the table
print(tables[0])

In [9]:
import pandas as pd
import numpy as np

def keep_best_rows(table):
    def row_mean(row):
        return row.xs('BP Mean', level='Metric', axis=0).mean()
    
    best_indices = []
    heuristics = table.index.get_level_values('Heuristic').unique()
    
    for heuristic in heuristics:
        heuristic_rows = table.xs(heuristic, level='Heuristic')
        best_row_index = heuristic_rows.apply(row_mean, axis=1).idxmin()
        best_indices.append((heuristic, best_row_index))
    
    # Create a new index from the best indices
    new_index = pd.MultiIndex.from_tuples(best_indices, names=['Heuristic', 'k'])
    
    # Use the new index to select the best rows from the original table
    filtered_table = table.loc[new_index]
    
    return filtered_table


keep_best_rows(tables[0])

In [10]:
import pandas as pd
import numpy as np

def format_table_for_publication_html(table):
    
    # Function to determine cell color
    def color_cell(value, min_value, min_std):
        if pd.isna(value):
            return ''
        if value <= min_value + min_std:
            return 'background-color: green; color: white'
        else:
            return 'background-color: red; color: white'

    #table = keep_best_rows(table)
    
    # Create a copy of the table with only BP Mean values
    mean_table = table.xs('BP Mean', axis=1, level='Metric')
    
    # Find the minimum value and its corresponding std for each load
    min_values = mean_table.min()
    min_indices = mean_table.idxmin()
    min_stds = pd.Series(index=min_values.index)
    for load in min_values.index:
        min_stds[load] = table.loc[min_indices[load], (load, 'BP Std')]
    
    # Apply coloring
    styled_table = mean_table.style.apply(lambda col: [color_cell(v, min_values[col.name], min_stds[col.name]) for v in col], axis=0)
    
    # Format numbers
    styled_table = styled_table.format("{:.4f}")
    
    # Add a caption
    #styled_table = styled_table.set_caption("Blocking Probability (BP) for different heuristics and loads. Green cells are within one standard deviation of the minimum BP for that load.")
    
    return styled_table

# Assuming 'tables' is your list of DataFrames, one for each topology
for i, table in enumerate(tables):
    styled_table = format_table_for_publication_html(table)
    
    # Display the table (in a Jupyter notebook this will show the formatted table)
    display(styled_table)
    
    # Save to HTML (you can then copy this into your paper or convert to LaTeX)
    styled_table.to_html(f'formatted_table_rmsa_{i}.html')

# If you need LaTeX output, you can use a library like pandas_to_latex or manually convert the HTML

# Plot the rows on a line chart of BP vs Load

In [95]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.scale import ScaleBase, register_scale
from matplotlib.transforms import Transform
from matplotlib.ticker import AutoLocator, ScalarFormatter

class LogLinearScale(ScaleBase):
    name = 'loglinear'

    def __init__(self, axis, **kwargs):
        ScaleBase.__init__(self, axis)
        self.thresh = kwargs.get('thresh', 0.1)

    def get_transform(self):
        return LogLinearTransform(self.thresh)

    def set_default_locators_and_formatters(self, axis):
        axis.set_major_locator(AutoLocator())
        axis.set_major_formatter(ScalarFormatter())

class LogLinearTransform(Transform):
    input_dims = output_dims = 1

    def __init__(self, thresh):
        Transform.__init__(self)
        self.thresh = thresh

    def transform_non_affine(self, a):
        mask = a < self.thresh
        with np.errstate(divide='ignore', invalid='ignore'):
            log_a = np.log10(a)
        out = np.where(mask, log_a, a / self.thresh + np.log10(self.thresh) - 1)
        return out

    def inverted(self):
        return InvertedLogLinearTransform(self.thresh)

class InvertedLogLinearTransform(Transform):
    input_dims = output_dims = 1

    def __init__(self, thresh):
        Transform.__init__(self)
        self.thresh = thresh

    def transform_non_affine(self, a):
        return np.where(a < np.log10(self.thresh),
                        10.0 ** a,
                        (a - np.log10(self.thresh) + 1) * self.thresh)

    def inverted(self):
        return LogLinearTransform(self.thresh)

# Register the custom scale
register_scale(LogLinearScale)


In [96]:
import matplotlib.pyplot as plt

convert_heuristic_name = lambda x: x[0].replace("K", f"{x[1]}-")


def plot_bp_vs_load(table, topology_name):
    plt.figure(figsize=(12, 8))
    
    # Extract BP Mean values
    bp_mean = table.xs('BP Mean', axis=1, level='Metric')
    bp_std = table.xs('BP Std', axis=1, level='Metric')
    
    # Get load values
    loads = bp_mean.columns.astype(int)
    
    # Plot each heuristic
    for heuristic in bp_mean.index:
        plt.plot(loads, bp_mean.loc[heuristic], marker='o', label=convert_heuristic_name(heuristic))
        # Plot std
        plt.fill_between(loads, bp_mean.loc[heuristic] - bp_std.loc[heuristic], bp_mean.loc[heuristic] + bp_std.loc[heuristic], alpha=0.2)

    # Get model eval data
    if topology_name == "NSFNET":
        model_eval = []
        for load in load_range:
            model_eval_file = f"/Users/michaeldoherty/git/XLRON/data/JOCN_SI/deeprmsa_model_eval/JOCN_DEEPRMSA_MASKED_8_1_{load}.csv"
            df = pd.read_csv(model_eval_file)
            mean_bp = df["service_blocking_probability"].mean()
            std_bp = np.sqrt(np.sum(df["service_blocking_probability_std"] ** 2) / len(df))
            model_eval.append([load, mean_bp, std_bp])
        
        model_eval_df = pd.DataFrame(model_eval, columns=['Load', 'BP Mean', 'BP Std'])
        plt.plot(model_eval_df['Load'], model_eval_df['BP Mean'], label='XLRON', marker='o')
        plt.fill_between(model_eval_df['Load'], model_eval_df['BP Mean'] - model_eval_df['BP Std'], model_eval_df['BP Mean'] + model_eval_df['BP Std'], alpha=0.2)
    
    plt.xlabel('Load (Erlangs)', fontsize=14)
    plt.ylabel('Blocking pprobability', fontsize=14)
    plt.xscale('linear')  # Use log scale for x-axis
    plt.yscale('log')  # Use log scale for y-axis
    plt.grid(True, which="both", ls="-", alpha=0.2)
    plt.legend(fontsize=10, loc='upper left')
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(f'bp_vs_load_{topology_name}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

# Assuming 'tables' is your list of DataFrames, one for each topology
topologies = ["NSFNET", "COST239"]  # Add your topology names here

for i, table in enumerate(tables):
    # Format and display the table
    table = keep_best_rows(table)
    styled_table = format_table_for_publication_html(table)
    display(styled_table)
    styled_table.to_html(f'formatted_table_rmsa_{i}.html')
    
    # Create the plot
    table = table.replace(0, np.nan)
    plot_bp_vs_load(table, topologies[i])

print("Tables and plots have been saved.")

In [97]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def convert_heuristic_name(x):
    return x[0].replace("K", f"{x[1]}-")

def plot_bp_vs_load(table, topology_name):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Extract BP Mean values
    bp_mean = table.xs('BP Mean', axis=1, level='Metric')
    bp_std = table.xs('BP Std', axis=1, level='Metric')
    
    # Get load values
    loads = bp_mean.columns.astype(int)

    legend_lines = {}
    # Plot each heuristic
    for heuristic in bp_mean.index:
        label = convert_heuristic_name(heuristic)
        # First subplot (0-250, log scale)
        mask1 = loads <= 250
        line, = ax1.plot(loads[mask1], bp_mean.loc[heuristic, mask1], marker='o', label=label)
        ax1.fill_between(loads[mask1], 
                         bp_mean.loc[heuristic, mask1] - bp_std.loc[heuristic, mask1], 
                         bp_mean.loc[heuristic, mask1] + bp_std.loc[heuristic, mask1], 
                         alpha=0.2)
        legend_lines[label] = line
        
        
        # Second subplot (250-800, linear scale)
        mask2 = loads > 250
        ax2.plot(loads[mask2], bp_mean.loc[heuristic, mask2], marker='o')
        ax2.fill_between(loads[mask2], 
                         bp_mean.loc[heuristic, mask2] - bp_std.loc[heuristic, mask2], 
                         bp_mean.loc[heuristic, mask2] + bp_std.loc[heuristic, mask2], 
                         alpha=0.2)

    # Get model eval data
    if topology_name == "NSFNET":
        model_eval = []
        for load in loads:
            model_eval_file = f"/Users/michaeldoherty/git/XLRON/data/JOCN_SI/deeprmsa_model_eval/JOCN_DEEPRMSA_MASKED_8_1_{load}.csv"
            df = pd.read_csv(model_eval_file)
            mean_bp = df["service_blocking_probability"].mean()
            std_bp = np.sqrt(np.sum(df["service_blocking_probability_std"] ** 2) / len(df))
            model_eval.append([load, mean_bp, std_bp])
        
        model_eval_df = pd.DataFrame(model_eval, columns=['Load', 'BP Mean', 'BP Std'])
        
        # First subplot
        mask1 = model_eval_df['Load'] <= 250
        line, = ax1.plot(model_eval_df.loc[mask1, 'Load'], model_eval_df.loc[mask1, 'BP Mean'], label='XLRON', marker='o')
        ax1.fill_between(model_eval_df.loc[mask1, 'Load'], 
                         model_eval_df.loc[mask1, 'BP Mean'] - model_eval_df.loc[mask1, 'BP Std'], 
                         model_eval_df.loc[mask1, 'BP Mean'] + model_eval_df.loc[mask1, 'BP Std'], 
                         alpha=0.2)
        legend_lines['XLRON']=line
        
        # Second subplot
        mask2 = model_eval_df['Load'] > 250
        ax2.plot(model_eval_df.loc[mask2, 'Load'], model_eval_df.loc[mask2, 'BP Mean'], marker='o')
        ax2.fill_between(model_eval_df.loc[mask2, 'Load'], 
                         model_eval_df.loc[mask2, 'BP Mean'] - model_eval_df.loc[mask2, 'BP Std'], 
                         model_eval_df.loc[mask2, 'BP Mean'] + model_eval_df.loc[mask2, 'BP Std'], 
                         alpha=0.2)
    
    # Configure subplots
    for ax in (ax1, ax2):
        ax.set_xlabel('Load (Erlangs)', fontsize=20)
        ax.grid(True, which="both", ls="-", alpha=0.2)
        ax.tick_params(axis='both', which='major', labelsize=18)
    
    ax1.set_xlim(0, 250)
    ax1.set_yscale('log')
    ax2.set_xlim(250, 800)
    ax1.set_ylim(1e-5, 0.2)
    ax2.set_ylim(0, 0.5)
    
    # Set y-label only for the left subplot
    ax1.set_ylabel('Blocking probability', fontsize=20)

    # Add legend to the top left of the first subplot with custom order
    legend_order = ["BF-6-SP", "3-CA-FF", "3-MF-FF", "10-SP-BF", "FF-4-SP", "10-ME-FF", "9-SP-FF", "5-MC-FF", "XLRON"]
    ordered_legend = [legend_lines[label] for label in legend_order if label in legend_lines]
    ax1.legend(ordered_legend, ["BF-6-SP", "3-CA-FF", "3-MF-FF", "10-SP-BF", "FF-4-SP", "10-ME-FF", "9-SP-FF", "5-MC-FF", "XLRON"], fontsize=16, loc='upper left')
    
    # Add legend to the top left of the first subplot
    #ax1.legend(fontsize=16, loc='upper left')
    
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(f'bp_vs_load_{topology_name}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

# The rest of your code remains the same
topologies = ["NSFNET", "COST239"]  # Add your topology names here

for i, table in enumerate(tables):
    # Format and display the table
    table = keep_best_rows(table)
    styled_table = format_table_for_publication_html(table)
    display(styled_table)
    styled_table.to_html(f'formatted_table_rmsa_{i}.html')
    
    # Create the plot
    table = table.replace(0, np.nan)
    plot_bp_vs_load(table, topologies[i])

print("Tables and plots have been saved.")



In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def format_table_for_publication(table):
    # Create a copy of the table with only BP Mean values
    mean_table = table.xs('BP Mean', axis=1, level='Metric')
    
    # Find the minimum value and its corresponding std for each load
    min_values = mean_table.min()
    min_indices = mean_table.idxmin()
    min_stds = pd.Series(index=min_values.index)
    for load in min_values.index:
        min_stds[load] = table.loc[min_indices[load], (load, 'BP Std')]
    
    return mean_table, min_values, min_stds

def create_styled_table_png(table, min_values, min_stds, filename, dpi=300):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(20, len(table) * 0.5))
    ax.axis('off')
    
    # Create the table
    table_obj = ax.table(cellText=table.values,
                         rowLabels=table.index,
                         colLabels=table.columns,
                         cellLoc='center',
                         loc='center')
    
    # Style the table
    table_obj.auto_set_font_size(False)
    table_obj.set_fontsize(9)
    table_obj.scale(1.2, 1.5)
    
    # Color the cells
    for (row, col), cell in table_obj.get_celld().items():
        if row == 0:  # Header
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        elif col == -1:  # Row labels
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        else:
            value = table.iloc[row-1, col]
            if value <= min_values[table.columns[col]] + min_stds[table.columns[col]]:
                cell.set_facecolor('#C6EFCE')
            else:
                cell.set_facecolor('#FFC7CE')
    
    # Add title
    plt.title('Blocking Probability (BP) for different heuristics and loads.\n'
              'Green cells are within one standard deviation of the minimum BP for that load.',
              fontsize=12, fontweight='bold', pad=20)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0.5)
    plt.close()

# Assuming 'tables' is your list of DataFrames, one for each topology
for i, table in enumerate(tables):
    mean_table, min_values, min_stds = format_table_for_publication(table)
    create_styled_table_png(mean_table, min_values, min_stds, f'formatted_table_rmsa_topology_{i}.png')
    print(f"PNG table for topology {i} has been saved to 'formatted_table_rmsa_topology_{i}.png'")

print("\nAll tables have been saved as PNG images.")

In [16]:
import pandas as pd
import numpy as np

def format_heuristic(heuristic):
    return heuristic.upper().replace('_', '-')

env_type = "rwa_lightpath_reuse"
topologies = ["cost239", "nsfnet"]
heuristics = ['ksp_ff', 'ff_ksp']#, 'ksp_mu']#, 'ksp_mu_nonrel', 'ksp_mu_unique', 'mu_ksp', 'mu_ksp_nonrel', 'mu_ksp_unique']
num_requests_nsfnet = [1e4, 1e4, 1.5e4, 2e4, 2.5e4]
num_requests_cost239 = [2e4, 2e4, 2.5e4, 3e4, 3.5e4]
k_range = range(1, 11)
tables = []

for topology in topologies:
    data = []
    for heuristic in heuristics:
        for k in k_range:
            row = []
            num_requests_list = num_requests_nsfnet if topology == "nsfnet" else num_requests_cost239
            for i, num_requests in enumerate(num_requests_list):
                first_blocking = True if i == 0 else False
                df = read_rwa_lightpath_reuse_files(topology, heuristic, k, num_requests, first_blocking=first_blocking)
                accepted_services_mean = df["accepted_services"].mean()
                accepted_services_std = np.sqrt(np.sum(df["accepted_services_std"]**2)/len(df))
                row.extend([accepted_services_mean, accepted_services_std])
            data.append([format_heuristic(heuristic), k] + row)

    # Create multi-index DataFrame with titled columns
    num_requests_list[0] = "First Block"
    columns = ['Heuristic', 'k']
    for num_requests in num_requests_list:
        num_requests = f"{num_requests/1000:.0f}k" if num_requests != "First Block" else num_requests
        columns.extend([(num_requests, 'Accepted Services Mean'), (num_requests, 'Accepted Services Std')])
    
    table = pd.DataFrame(data, columns=columns)
    table = table.set_index(['Heuristic', 'k'])
    
    # Create a MultiIndex for the columns
    table.columns = pd.MultiIndex.from_tuples(table.columns, names=['# Requests', 'Metric'])
    
    tables.append(table)

# Example of how to display the table
print(tables[0])

In [17]:
import pandas as pd
import numpy as np

def format_table_for_publication(table):
    # Function to determine cell color
    
    def color_cell(value, max_value, max_std):
        if pd.isna(value):
            return ''
        if value >= max_value - max_std:
            return 'background-color: green; color: white'
        else:
            return 'background-color: red; color: white'
    
    # Create a copy of the table with only BP Mean values
    mean_table = table.xs('Accepted Services Mean', axis=1, level='Metric')
    
    # Find the minimum value and its corresponding std for each load
    max_values = mean_table.max()
    max_indices = mean_table.idxmax()
    max_stds = pd.Series(index=max_values.index)
    for num_requests in max_values.index:
        max_stds[num_requests] = table.loc[max_indices[num_requests], (num_requests, 'Accepted Services Std')]
    
    # Apply coloring
    styled_table = mean_table.style.apply(lambda col: [color_cell(v, max_values[col.name], max_stds[col.name]) for v in col], axis=0)
    
    # Format numbers
    styled_table = styled_table.format("{:.0f}")
    
    # Add a caption
    #styled_table = styled_table.set_caption("Blocking Probability (BP) for different heuristics and loads. Green cells are within one standard deviation of the minimum BP for that load.")
    
    return styled_table

# Assuming 'tables' is your list of DataFrames, one for each topology
for i, table in enumerate(tables):
    styled_table = format_table_for_publication(table)
    
    # Display the table (in a Jupyter notebook this will show the formatted table)
    display(styled_table)
    
    # Save to HTML (you can then copy this into your paper or convert to LaTeX)
    styled_table.to_html(f'formatted_table_rwalr_{i}.html')

# If you need LaTeX output, you can use a library like pandas_to_latex or manually convert the HTML

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def format_table_for_publication(table):
    # Create a copy of the table with only BP Mean values
    mean_table = table.xs('BP Mean', axis=1, level='Metric')
    
    # Find the minimum value and its corresponding std for each load
    min_values = mean_table.min()
    min_indices = mean_table.idxmin()
    min_stds = pd.Series(index=min_values.index)
    for load in min_values.index:
        min_stds[load] = table.loc[min_indices[load], (load, 'BP Std')]
    
    return mean_table, min_values, min_stds

def create_styled_table_png(table, min_values, min_stds, filename, dpi=300):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(20, len(table) * 0.5))
    ax.axis('off')
    
    # Create the table
    table_obj = ax.table(cellText=table.values,
                         rowLabels=table.index,
                         colLabels=table.columns,
                         cellLoc='center',
                         loc='center')
    
    # Style the table
    table_obj.auto_set_font_size(False)
    table_obj.set_fontsize(9)
    table_obj.scale(1.2, 1.5)
    
    # Color the cells
    for (row, col), cell in table_obj.get_celld().items():
        if row == 0:  # Header
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        elif col == -1:  # Row labels
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        else:
            value = table.iloc[row-1, col]
            if value <= min_values[table.columns[col]] + min_stds[table.columns[col]]:
                cell.set_facecolor('#C6EFCE')
            else:
                cell.set_facecolor('#FFC7CE')
    
    # Add title
    plt.title('Blocking Probability (BP) for different heuristics and loads.\n'
              'Green cells are within one standard deviation of the minimum BP for that load.',
              fontsize=12, fontweight='bold', pad=20)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0.5)
    plt.close()

# Assuming 'tables' is your list of DataFrames, one for each topology
for i, table in enumerate(tables):
    mean_table, min_values, min_stds = format_table_for_publication(table)
    create_styled_table_png(mean_table, min_values, min_stds, f'formatted_table_rwalr_topology_{i}.png')
    print(f"PNG table for topology {i} has been saved to 'formatted_table_rwalr_topology_{i}.png'")

print("\nAll tables have been saved as PNG images.")

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import numpy as np
# 
# def plot_blocking_prob_vs_load(table, heuristics, k_values, filename='blocking_prob_vs_load.png'):
#     plt.figure(figsize=(12, 8))
#     for heuristic in heuristics:
#         for k in k_values:
#             data = table.loc[heuristic, (k, slice(None), 'mean')]
#             plt.plot(load_range, data.values, label=f'{heuristic}, k={k}', marker='o')
#     
#     plt.xlabel('Load')
#     plt.ylabel('Blocking Probability')
#     plt.title('Blocking Probability vs Load for Different Heuristics and k Values')
#     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.tight_layout()
#     plt.show()
# 
# def plot_heatmap(table, filename='blocking_prob_heatmap.png'):
#     mean_data = table.xs('mean', level='BP', axis=1)
#     plt.figure(figsize=(15, 10))
#     sns.heatmap(mean_data, cmap='YlOrRd', annot=True, fmt='.2e', cbar_kws={'label': 'Blocking Probability'})
#     plt.title('Heatmap of Blocking Probability for Different Heuristics, k Values, and Loads')
#     plt.tight_layout()
#     plt.show()
# 
# def plot_box_plot(table, filename='blocking_prob_box_plot.png'):
#     # Select only the 'mean' values and reset the index
#     data_mean = table.xs('mean', level='BP', axis=1).stack().reset_index()
#     data_mean.columns = ['Heuristic', 'k', 'load', 'Blocking Probability']
#     
#     plt.figure(figsize=(15, 8))
#     sns.boxplot(x='Heuristic', y='Blocking Probability', data=data_mean)
#     plt.xticks(rotation=45)
#     plt.title('Distribution of Blocking Probability for Different Heuristics')
#     plt.tight_layout()
#     plt.show()
# 
# # Usage
# for i, table in enumerate(tables):
#     plot_blocking_prob_vs_load(table, heuristics[:3], [1, 5, 10], f'blocking_prob_vs_load_{topologies[i]}.png')
#     plot_heatmap(table, f'blocking_prob_heatmap_{topologies[i]}.png')
#     plot_box_plot(table, f'blocking_prob_box_plot_{topologies[i]}.png')
# 
# # For comparing topologies:
# fig, axes = plt.subplots(1, 2, figsize=(20, 8))
# for i, topology in enumerate(topologies):
#     mean_data = tables[i].xs('mean', level='BP', axis=1)
#     sns.heatmap(mean_data, cmap='YlOrRd', annot=False, cbar_kws={'label': 'Blocking Probability'}, ax=axes[i])
#     axes[i].set_title(f'Blocking Probability Heatmap for {topology}')
# plt.tight_layout()
# plt.show()